# Planet Tasking API Order Lock-In

---

## Introduction

---

This tutorial is an introduction on how to create locked-in orders using [Planet](https://www.planet.com)'s Tasking API. It provides code samples on how to write simple Python code to do this.

The API reference documentation can be found at https://developers.planet.com/docs/tasking

### Requirements

---

#### Software & Modules

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Familiarity with basic REST API concepts and usage is also assumed.

We'll be using a **"Jupyter Notebook"** (aka Python Notebook) to run through the examples.
To learn more about and get started with using Jupyter, visit: [Jupyter](https://jupyter.org/) and [IPython](https://ipython.org/). 

For the best experience, download this notebook and run it on your system, and make sure to install the modules listed below first. You can also copy the examples' code to a separate Python files an run them directly with Python on your system if you prefer.

#### Planet API Key

You should have an account on the Planet Platform to access the Tasking API. You may retrieve your API key from your [account page](https://www.planet.com/account/), or from the "API Tab" in [Planet Explorer](https://www.planet.com/explorer).

## Overview

---

### The basic workflow

1. Choose an imaging window
1. Create a locked-in order
1. Check the progress of the order

### API Endpoints

This tutorial will cover the following API ***endpoint***:

* [`/order`](https://api.planet.com/tasking/v2/order/)

## Basic Setup

---

Before interacting with the Planet Tasking API using Python, we will set up our environment with some useful modules and helper functions.

* We'll configure *authentication* to the Planet Tasking API
* We'll use the `requests` Python module to make HTTP communication easier. 
* We'll use the `json` Python module to help us work with JSON responses from the API.
* We'll use the `pytz` Python module to define the time frame for the order that we will be creating.
* We'll create a function called `p` that will print Python dictionaries nicely.

Then we'll be ready to make our first call to the Planet Tasking API by hitting the base endpoint at `https://api.planet.com/tasking/v2`. 

Let's start by configuring authentication:

### Authentication

Authentication with the Planet Tasking API can be achieved using a valid Planet **API key**.

You can *export* your API Key as an environment variable on your system:

`export PL_API_KEY="YOUR API KEY HERE"`

Or add the variable to your path, etc.

To start our Python code, we'll setup an API Key variable from an environment variable to use with our requests:



### Helper Modules and Functions

In [ ]:
# Import helper modules
import json
import requests
import pytz
from time import sleep
from datetime import datetime, timedelta

In [ ]:
# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

In [ ]:
# Setup Planet Tasking PLANET_API_HOST
TASKING_API_URL = "https://api.planet.com/tasking/v2"

# Setup the session
session = requests.Session()

# Authenticate
session.headers.update({
    'Authorization': f'api-key {PLANET_API_KEY}',
    'Content-Type': 'application/json'
})

## 1 | Choosing an imaging window

We want to create a locked-in tasking order which will take an image of a specific location within a time window picked from provided range of available windows that lie within the provided time of interest. To do this, we first need to query the /imaging-windows endpoint of the Tasking API, providing to it the same coordinatea and time range that will be used for the creation of the tasking order.

To make things easier, we will default the start and end time to start tomorrow and end 7 days from now. Of course, feel free to change this to suit your needs, but if you do, take note that all times should be in UTC format. Unlike with a standard, non-locked-in tasking order, the start and end time are required so as to provide the imaging windows service a time frame in which to search. The start-time for a locked-in order **must** be at least 12 hours in the future

In [ ]:
latitude=float(input("Provide the latitude"))
longitude=float(input("Provide the longitude"))

# Set a start and end time, giving the order a month to complete
tomorrow = datetime.now(pytz.utc) + timedelta(days=1)
seven_days_later = tomorrow + timedelta(days=7)

window_request = {
	"geometry": {
		"coordinates": [
            longitude,
            latitude
		],
		"type": "Point"
	},
	"startTime": tomorrow.isoformat(),
    "endTime": seven_days_later.isoformat(),
}

In [ ]:
# An imaging windows request is a POST request to the /imaging-windows endpoint
res = session.request('POST', TASKING_API_URL + '/imaging-windows', json=window_request)

if res.status_code == 403:
    print('Your PLANET_API_KEY is valid, but you are not authorized.')
elif res.status_code == 401:
    print('Your PLANET_API_KEY is incorrect')
elif res.status_code == 201:
    print('Your imaging windows request was successful')
else:
    print(f'Received status code {res.status_code} from the API. Please contact support.')

# View the Location header in the response
print(res.headers['Location'])

The response body is empty, but for an imaging windows request, the **Location** header is the important part. This contains a URL with an ID which, when part of a GET request, will return the actual array of imaging windows.

In [ ]:
location = res.headers['Location']

res = session.request('GET', 'https://api.planet.com' + location)

if res.status_code == 403:
    print('Your PLANET_API_KEY is valid, but you are not authorized.')
elif res.status_code == 401:
    print('Your PLANET_API_KEY is incorrect')
elif res.status_code == 201:
    print('Your imaging windows have been returned successfully')
else:
    print(f'Received status code {res.status_code} from the API. Please contact support.')

p(res.json())

## 2 | Creating the locked-in order

Now that we have the list of available imaging windows, we need to select one and use it to create the locked-in order. If the returned array is empty, that will be because the provided coodinates/time of interest combination does not have any available windows. If this happens, change either the location or the time and try again.

As mentioned already, the same coordinates and timeframe that were used for requesting the imaging windows will be used in the creation of the order. Changing these will result in the order not being fulfilled.

In [ ]:
imaging_windows=res.json()['results']

# We are just going to take the first window in the list - of course you should change this to pick a window that actually suits your needs
imaging_window=imaging_windows[0]['id']

In [ ]:
# Define the name and coordinates for the order
name=input("Give the order a name")

# Because the geometry is GeoJSON, the coordinates must be longitude,latitude
order = {
    'name': name,
    'geometry': {
        'type': 'Point',
        'coordinates': [
            longitude,
            latitude
        ]
    },
	"startTime": tomorrow.isoformat(),
    "endTime": seven_days_later.isoformat(),
    "imaging_window": imaging_window
}


In [ ]:
#View the payload before posting
p(order)

In [ ]:
# The creation of an order is a POST request to the /orders endpoint
res = session.request('POST', TASKING_API_URL + '/orders/', json=order)

if res.status_code == 403:
    print('Your PLANET_API_KEY is valid, but you are not authorized.')
elif res.status_code == 401:
    print('Your PLANET_API_KEY is incorrect')
elif res.status_code == 201:
    print('Your order was created successfully')
else:
    print(f'Received status code {res.status_code} from the API. Please contact support.')

# View the response
p(res.json())

## 2 | Check the status of the monitoring order

To see the status of an existing locked-in order, the tasking order id is required. Depending on the tasking order, it can take some time for the status of the tasking order to change, and so you may need to come back to this section once some time has elapsed before changes to the tasking order can be seen. It is recommended to run the next part of this notebook to extract the ID of the newly created order and save that for later use.

In [ ]:
# Get the response JSON and extract the ID of the order
response = res.json()
new_order_id = response["id"]
p(new_order_id)

In [ ]:
def check_order_status(order_id):
    # Make a GET request with the order_id concatenated to the end of the /orders url; e.g. https://api.planet.com/tasking/v2/orders/<ORDER_ID>
    res = session.request('GET', TASKING_API_URL + '/orders/' + order_id)

    if res.status_code == 403:
        print('Your PLANET_API_KEYPLANET_API_KEY is valid, but you are not authorized to view this order.')
    elif res.status_code == 401:
        print('Your PLANET_API_KEYPLANET_API_KEY is incorrect')
    elif res.status_code == 404:
        print(f'Your order ({order_id}) does not exist')
    elif res.status_code != 200:
        print(f'Received status code {res.status_code} from the API. Please contact support.')
    else:
        order = res.json()
        p(res.json())
        print(f'Your order is {order["status"]} with {order["capture_status_published_count"]} published captures '
                f'and {order["capture_assessment_success_count"]} successful captures')

In [ ]:
check_order_status(new_order_id)